# Phase 3: Specialists
- Auto-create specialists from topic labels.
- Run short adaptation loops per topic/batch.

In [ ]:
%pip install -q torch datasets transformers sentencepiece tqdm
import torch, sys
from pathlib import Path
ROOT = Path('/content/repo') if Path('/content/repo').exists() else Path.cwd().parent
sys.path.insert(0, str(ROOT))
from colab_l4_training import get_test_config, main
from core.specialist import SpecialistRegistry

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Load model
config = get_test_config()
config.enable_continuous_learning = False
model, losses, cl_orch = main(config_preset='test')
hippocampus = model.hippocampus

+registry = SpecialistRegistry(n_features=config.embedding_dim).to(device)
topics = [
    'history', 'physics', 'philosophy', 'medicine', 'math',
    'technology', 'politics', 'art', 'music', 'sports'
]
specs = registry.ensure_from_topics(topics)
print('Specialists created:', list(specs.keys()))

In [ ]:
# Toy adaptation loop per topic (placeholder)
optimizer = torch.optim.Adam(registry.parameters(), lr=1e-3)
for name, spec in registry.all.items():
    spec.train()
    x = torch.randn(4, 1, config.embedding_dim, device=device)  # fake embeddings
    out = spec(x)
    loss = out.pow(2).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Trained specialist {name}, loss={loss.item():.4f}")